In [7]:
import random
import numpy as np
import math
import 

In [139]:
import random

In [143]:
quad = lambda x: (np.sum(x ** 2), x * 2)

In [151]:
a = np.array([[1,2,3],[2,3,4]])
a

array([[1, 2, 3],
       [2, 3, 4]])

In [157]:
np.sum(a, axis = 1)

array([6, 9])

In [144]:
np.random.randn(3,)

array([-1.66417299, -0.12656933, -0.36343778])

In [145]:
gradcheck_naive(quad, np.random.randn(3,)) 

NameError: name 'gradcheck_naive' is not defined

In [142]:
rndstate = random.getstate()
rndstate

(3,
 (2147483648,
  1436410458,
  270803804,
  2147636634,
  2148224092,
  1135766104,
  1827825679,
  1799616059,
  2196190066,
  2062687492,
  2915089137,
  4048978017,
  2241308909,
  2019289412,
  491011790,
  279328983,
  3528291258,
  2507175406,
  2325736570,
  3126049241,
  611342437,
  4184221976,
  3004158443,
  1949789896,
  867358609,
  1343853161,
  4261816889,
  1508705461,
  1353417216,
  861522893,
  1189891626,
  3558619101,
  4081331007,
  80899469,
  2559323270,
  1440426207,
  2982405591,
  3679121875,
  219428089,
  1874771825,
  3769459308,
  3719100634,
  2034769342,
  863475456,
  1835887979,
  1861063001,
  706575001,
  3461734798,
  546213280,
  51941263,
  4264425263,
  2386429709,
  1027816741,
  1475550685,
  3929266193,
  2484916563,
  3311626326,
  3967398053,
  552031930,
  353158596,
  2412777824,
  1054905861,
  3702670871,
  1839974738,
  3786612062,
  1456457410,
  3457111839,
  1717397918,
  3019380148,
  2070469797,
  2216797990,
  343284274,
  344

In [8]:
def sigmoid(x): 
    return 1. / (1 + np.exp(-x))

In [9]:
def sigmoid_derivative(values): 
    return values*(1-values)

In [10]:
def tanh_derivative(values): 
    return 1. - values ** 2

In [11]:
# createst uniform random array w/ values in [a,b) and shape args
def rand_arr(a, b, *args): 
    np.random.seed(0)
    return np.random.rand(*args) * (b - a) + a

In [12]:
class LstmParam:
    def __init__(self, mem_cell_ct, x_dim):
        self.mem_cell_ct = mem_cell_ct
        self.x_dim = x_dim
        concat_len = x_dim + mem_cell_ct
        # weight matrices
        self.wg = rand_arr(-0.1, 0.1, mem_cell_ct, concat_len)
        self.wi = rand_arr(-0.1, 0.1, mem_cell_ct, concat_len) 
        self.wf = rand_arr(-0.1, 0.1, mem_cell_ct, concat_len)
        self.wo = rand_arr(-0.1, 0.1, mem_cell_ct, concat_len)
        # bias terms
        self.bg = rand_arr(-0.1, 0.1, mem_cell_ct) 
        self.bi = rand_arr(-0.1, 0.1, mem_cell_ct) 
        self.bf = rand_arr(-0.1, 0.1, mem_cell_ct) 
        self.bo = rand_arr(-0.1, 0.1, mem_cell_ct) 
        # diffs (derivative of loss function w.r.t. all parameters)
        self.wg_diff = np.zeros((mem_cell_ct, concat_len)) 
        self.wi_diff = np.zeros((mem_cell_ct, concat_len)) 
        self.wf_diff = np.zeros((mem_cell_ct, concat_len)) 
        self.wo_diff = np.zeros((mem_cell_ct, concat_len)) 
        self.bg_diff = np.zeros(mem_cell_ct) 
        self.bi_diff = np.zeros(mem_cell_ct) 
        self.bf_diff = np.zeros(mem_cell_ct) 
        self.bo_diff = np.zeros(mem_cell_ct) 

    def apply_diff(self, lr = 1):
        self.wg -= lr * self.wg_diff
        self.wi -= lr * self.wi_diff
        self.wf -= lr * self.wf_diff
        self.wo -= lr * self.wo_diff
        self.bg -= lr * self.bg_diff
        self.bi -= lr * self.bi_diff
        self.bf -= lr * self.bf_diff
        self.bo -= lr * self.bo_diff
        # reset diffs to zero
        self.wg_diff = np.zeros_like(self.wg)
        self.wi_diff = np.zeros_like(self.wi) 
        self.wf_diff = np.zeros_like(self.wf) 
        self.wo_diff = np.zeros_like(self.wo) 
        self.bg_diff = np.zeros_like(self.bg)
        self.bi_diff = np.zeros_like(self.bi) 
        self.bf_diff = np.zeros_like(self.bf) 
        self.bo_diff = np.zeros_like(self.bo) 

In [13]:
class LstmState:
    def __init__(self, mem_cell_ct, x_dim):
        self.g = np.zeros(mem_cell_ct)
        self.i = np.zeros(mem_cell_ct)
        self.f = np.zeros(mem_cell_ct)
        self.o = np.zeros(mem_cell_ct)
        self.s = np.zeros(mem_cell_ct)
        self.h = np.zeros(mem_cell_ct)
        self.bottom_diff_h = np.zeros_like(self.h)
        self.bottom_diff_s = np.zeros_like(self.s)

In [14]:
class LstmNode:
    def __init__(self, lstm_param, lstm_state):
        # store reference to parameters and to activations
        self.state = lstm_state
        self.param = lstm_param
        # non-recurrent input concatenated with recurrent input
        self.xc = None
    def bottom_data_is(self, x, s_prev = None, h_prev = None):
        # if this is the first lstm node in the network
        if s_prev is None: s_prev = np.zeros_like(self.state.s)
        if h_prev is None: h_prev = np.zeros_like(self.state.h)
        # save data for use in backprop
        self.s_prev = s_prev
        self.h_prev = h_prev

        # concatenate x(t) and h(t-1)
        xc = np.hstack((x,  h_prev))
        self.state.g = np.tanh(np.dot(self.param.wg, xc) + self.param.bg)
        self.state.i = sigmoid(np.dot(self.param.wi, xc) + self.param.bi)
        self.state.f = sigmoid(np.dot(self.param.wf, xc) + self.param.bf)
        self.state.o = sigmoid(np.dot(self.param.wo, xc) + self.param.bo)
        self.state.s = self.state.g * self.state.i + s_prev * self.state.f
        self.state.h = self.state.s * self.state.o

        self.xc = xc
    
    def top_diff_is(self, top_diff_h, top_diff_s):
        # notice that top_diff_s is carried along the constant error carousel
        ds = self.state.o * top_diff_h + top_diff_s
        do = self.state.s * top_diff_h
        di = self.state.g * ds
        dg = self.state.i * ds
        df = self.s_prev * ds

        # diffs w.r.t. vector inside sigma / tanh function
        di_input = sigmoid_derivative(self.state.i) * di 
        df_input = sigmoid_derivative(self.state.f) * df 
        do_input = sigmoid_derivative(self.state.o) * do 
        dg_input = tanh_derivative(self.state.g) * dg

        # diffs w.r.t. inputs
        self.param.wi_diff += np.outer(di_input, self.xc)
        self.param.wf_diff += np.outer(df_input, self.xc)
        self.param.wo_diff += np.outer(do_input, self.xc)
        self.param.wg_diff += np.outer(dg_input, self.xc)
        self.param.bi_diff += di_input
        self.param.bf_diff += df_input       
        self.param.bo_diff += do_input
        self.param.bg_diff += dg_input       

        # compute bottom diff
        dxc = np.zeros_like(self.xc)
        dxc += np.dot(self.param.wi.T, di_input)
        dxc += np.dot(self.param.wf.T, df_input)
        dxc += np.dot(self.param.wo.T, do_input)
        dxc += np.dot(self.param.wg.T, dg_input)

        # save bottom diffs
        self.state.bottom_diff_s = ds * self.state.f
        self.state.bottom_diff_h = dxc[self.param.x_dim:]

In [134]:
class LstmNetwork():
    def __init__(self, lstm_param):
        self.lstm_param = lstm_param
        self.lstm_node_list = []
        # input sequence
        self.x_list = []

    def y_list_is(self, y_list, loss_layer):
        """
        Updates diffs by setting target sequence 
        with corresponding loss layer. 
        Will *NOT* update parameters.  To update parameters,
        call self.lstm_param.apply_diff()
        """
        assert len(y_list) == len(self.x_list)
        idx = len(self.x_list) - 1
        # first node only gets diffs from label ...
        loss = loss_layer.loss(self.lstm_node_list[idx].state.h, y_list[idx])
        diff_h = loss_layer.bottom_diff(self.lstm_node_list[idx].state.h, y_list[idx])
        # here s is not affecting loss due to h(t+1), hence we set equal to zero
        diff_s = np.zeros(self.lstm_param.mem_cell_ct)
        self.lstm_node_list[idx].top_diff_is(diff_h, diff_s)
        idx -= 1

        ### ... following nodes also get diffs from next nodes, hence we add diffs to diff_h
        ### we also propagate error along constant error carousel using diff_s
        while idx >= 0:
            loss += loss_layer.loss(self.lstm_node_list[idx].state.h, y_list[idx])
            diff_h = loss_layer.bottom_diff(self.lstm_node_list[idx].state.h, y_list[idx])
            diff_h += self.lstm_node_list[idx + 1].state.bottom_diff_h
            diff_s = self.lstm_node_list[idx + 1].state.bottom_diff_s
            self.lstm_node_list[idx].top_diff_is(diff_h, diff_s)
            idx -= 1 

        return loss

    def x_list_clear(self):
        self.x_list = []

    def x_list_add(self, x):
        self.x_list.append(x)
        if len(self.x_list) > len(self.lstm_node_list):
            # need to add new lstm node, create new state mem
            lstm_state = LstmState(self.lstm_param.mem_cell_ct, self.lstm_param.x_dim)
            print(lstm_state.h)
            self.lstm_node_list.append(LstmNode(self.lstm_param, lstm_state))

        # get index of most recent x input
        idx = len(self.x_list) - 1
        if idx == 0:
            # no recurrent inputs yet
            self.lstm_node_list[idx].bottom_data_is(x)
        else:
            s_prev = self.lstm_node_list[idx - 1].state.s
            h_prev = self.lstm_node_list[idx - 1].state.h
            self.lstm_node_list[idx].bottom_data_is(x, s_prev, h_prev)


In [16]:
import numpy as np
from lstm import LstmParam,LstmNetwork

In [17]:
class ToyLossLayer:
    """
    Computes square loss with first element of hidden layer array.
    """
    @classmethod
    def loss(self, pred, label):
        return (pred[0] - label) ** 2

    @classmethod
    def bottom_diff(self, pred, label):
        diff = np.zeros_like(pred)
        diff[0] = 2 * (pred[0] - label)
        return diff

In [18]:
def example_0():
    # learns to repeat simple sequence from random inputs
    np.random.seed(0)

    # parameters for input data dimension and lstm cell count
    mem_cell_ct = 100
    x_dim = 50
    lstm_param = LstmParam(mem_cell_ct, x_dim)
    lstm_net = LstmNetwork(lstm_param)
    y_list = [-0.5, 0.2, 0.1, -0.5]
    input_val_arr = [np.random.random(x_dim) for _ in y_list]

    for cur_iter in range(100):
        print("iter", "%2s" % str(cur_iter), end=": ")
        for ind in range(len(y_list)):
            lstm_net.x_list_add(input_val_arr[ind])

        print("y_pred = [" +
              ", ".join(["% 2.5f" % lstm_net.lstm_node_list[ind].state.h[0] for ind in range(len(y_list))]) +
              "]", end=", ")

        loss = lstm_net.y_list_is(y_list, ToyLossLayer)
        print("loss:", "%.3e" % loss)
        lstm_param.apply_diff(lr=0.1)
        lstm_net.x_list_clear()

In [56]:
mem_cell_ct = 100
x_dim = 50
lstm_param = LstmParam(mem_cell_ct, x_dim)

print('Number of cell is',mem_cell_ct)
print('Dimension of input is',x_dim)

print('Shape of wg is',np.shape(lstm_param.wg))
print('Shape of wi is',np.shape(lstm_param.wi))
print('Shape of wf is',np.shape(lstm_param.wf))
print('Shape of wo is',np.shape(lstm_param.wo))

print('Shape of bg is',np.shape(lstm_param.bg))
print('Shape of bi is',np.shape(lstm_param.bi))
print('Shape of bf is',np.shape(lstm_param.bf))
print('Shape of bo is',np.shape(lstm_param.bo))

print('Shape of wg_diff is',np.shape(lstm_param.wg_diff))
print('Shape of wi_diff is',np.shape(lstm_param.wi_diff))
print('Shape of wf_diff is',np.shape(lstm_param.wf_diff))
print('Shape of wo_diff is',np.shape(lstm_param.wo_diff))

print('Shape of bg_diff is',np.shape(lstm_param.bg_diff))
print('Shape of bi_diff is',np.shape(lstm_param.bi_diff))
print('Shape of bf_diff is',np.shape(lstm_param.bf_diff))
print('Shape of bo_diff is',np.shape(lstm_param.bo_diff))

Number of cell is 100
Dimension of input is 50
Shape of wg is (100, 150)
Shape of wi is (100, 150)
Shape of wf is (100, 150)
Shape of wo is (100, 150)
Shape of bg is (100,)
Shape of bi is (100,)
Shape of bf is (100,)
Shape of bo is (100,)
Shape of wg_diff is (100, 150)
Shape of wi_diff is (100, 150)
Shape of wf_diff is (100, 150)
Shape of wo_diff is (100, 150)
Shape of bg_diff is (100,)
Shape of bi_diff is (100,)
Shape of bf_diff is (100,)
Shape of bo_diff is (100,)


In [128]:
lstm_net = LstmNetwork(lstm_param)
lstm_net.lstm_node_list

[]

In [129]:
len(lstm_net.lstm_node_list)

0

In [130]:
lstm_net.x_list

[]

In [132]:
y_list = [-0.5, 0.2, 0.1, -0.5]

In [133]:
input_val_arr = [np.random.random(x_dim) for _ in y_list]
np.shape(input_val_arr)

(4, 50)

In [135]:
lstm_net.x_list_add(input_val_arr[0])

In [136]:
len(lstm_net.x_list)

1

In [137]:
lstm_net.lstm_node_list

In [138]:
lstm_net.lstm_node_list[0].state.h

array([ 0.01738188, -0.008332  , -0.06584833,  0.02343224,  0.00613621,
        0.00050119, -0.0326001 , -0.04183327, -0.01478104,  0.0403268 ,
        0.09509385, -0.04515467,  0.11344356,  0.04130644, -0.06479724,
        0.01616473, -0.02901481, -0.02486891, -0.03781047,  0.08279321,
        0.01321319, -0.04145167,  0.02760036,  0.0103368 , -0.05075837,
        0.16915936, -0.06728949,  0.00982914,  0.19769348, -0.02076124,
       -0.03506644, -0.05793739,  0.00145878, -0.06344969, -0.03127233,
        0.09343004,  0.08725885,  0.07332262, -0.04414092, -0.05023508,
        0.09465628,  0.04262281,  0.00294111, -0.04238227, -0.06565515,
        0.14525706, -0.05345602,  0.04636952, -0.01923774,  0.10335337,
       -0.05383628, -0.04118786,  0.17998834,  0.02031099,  0.05047418,
       -0.03693532,  0.13210225,  0.0184269 ,  0.08767573, -0.02632053,
        0.00156274,  0.03508814,  0.10492902, -0.04816728,  0.01140923,
        0.20003025,  0.00797456, -0.03519422,  0.00392615, -0.06

In [105]:
for ind in range(len(y_list)):
    lstm_net.x_list_add(input_val_arr[ind])
    print(np.shape(lstm_net.x_list))

(1, 50)
(2, 50)
(3, 50)
(4, 50)


In [109]:
lstm_net.lstm_node_list

In [110]:
print("y_pred = [" +
              ", ".join(["% 2.5f" % lstm_net.lstm_node_list[ind].state.h[0] for ind in range(len(y_list))]) +
              "]", end=", ")

y_pred = [ 0.03019,  0.07607,  0.09577,  0.07669], 

In [113]:
lstm_net.lstm_node_list[1].state.h[0]

0.076069967106704467

In [115]:
np.shape(input_val_arr[0])

(50,)